# Random Forest

Trataremos de predecir si un cliente pagará su préstamo o no. Este es un problema de clasificación binaria donde los árboles de decisión pueden ayudarnos a tomar decisiones claras basadas en reglas. El conjunto de datos que vamos a utilizar es el de "Credit Card Default" de sklearn.datasets.

## Paso 1: Importar las librerías necesarias

Primero, importaremos las librerías y cargaremos los datos necesarios para nuestro ejemplo.

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# Cargar el dataset de default de crédito
data = fetch_openml(name='credit-g', version=1, as_frame=True)
df = data.frame

## Paso 2: Exploración de los datos

Revisaremos la estructura de nuestro conjunto de datos. Esto con el objetivo de identificar si debemos o no, ejecutar una fase de limpieza y/o estandarización de variables para facilitar la implementación de nuestro modelo.

In [4]:
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,4,male single,none,...,real estate,67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,2,female div/dep/mar,none,...,real estate,22,none,own,1,skilled,1,none,yes,bad
2,no checking,12,critical/other existing credit,education,2096,<100,4<=X<7,2,male single,none,...,real estate,49,none,own,1,unskilled resident,2,none,yes,good
3,<0,42,existing paid,furniture/equipment,7882,<100,4<=X<7,2,male single,guarantor,...,life insurance,45,none,for free,1,skilled,2,none,yes,good
4,<0,24,delayed previously,new car,4870,<100,1<=X<4,3,male single,none,...,no known property,53,none,for free,2,skilled,2,none,yes,bad


In [5]:
df.dtypes

checking_status           category
duration                     int64
credit_history            category
purpose                   category
credit_amount                int64
savings_status            category
employment                category
installment_commitment       int64
personal_status           category
other_parties             category
residence_since              int64
property_magnitude        category
age                          int64
other_payment_plans       category
housing                   category
existing_credits             int64
job                       category
num_dependents               int64
own_telephone             category
foreign_worker            category
class                     category
dtype: object

Podemos observar que existen variables que dificultarán la implementación del modelo. Por este motivo, será necesario implementar la siguiente fase de preprocesamiento:

## Paso 3: Preprocesamiento de datos
Vamos a organizar nuestros datos dividiéndolos en características (inputs) y etiquetas (targets). En este caso, estamos prediciendo una variable binaria, si un cliente "cumple" o "no cumple" con sus pagos.

In [ ]:
# Dividir datos en variables predictoras (X) y la variable objetivo (y)
X = df.drop(columns=['class'])
y = df['class']

# Convertir las etiquetas a valores numéricos (cumple=1, no cumple=0)
y = y.map({'good': 1, 'bad': 0})

In [16]:
# Uso de etiquetas (encoding)
X['checking_status'] = X['checking_status'].astype(str)
X.loc[df['checking_status'] == 'no checking', 'checking_status'] = 0
X.loc[df['checking_status'] == '<0', 'checking_status'] = 1
X.loc[df['checking_status'] == '0<=X<200', 'checking_status'] = 2
X.loc[df['checking_status'] == '>=200', 'checking_status'] = 3

In [17]:
X['credit_history'].value_counts()

credit_history
existing paid                     530
critical/other existing credit    293
delayed previously                 88
all paid                           49
no credits/all paid                40
Name: count, dtype: int64

¿Cómo podemos acelerar este proceso?

### Opción 1: Usar OneHotEncoder de sklearn

In [23]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Seleccionar las columnas categóricas
categorical_cols = X.select_dtypes(include=['category']).columns

# Crear el transformador para hacer el encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # Deja las columnas numéricas sin cambios
)

# Aplicar el transformador
X_encoded = preprocessor.fit_transform(X)

# Convertir a DataFrame y mostrar las primeras filas
X_encoded_df = pd.DataFrame(X_encoded, columns=preprocessor.get_feature_names_out())
X_encoded_df.head()

,cat__credit_history_all paid,cat__credit_history_critical/other existing credit,cat__credit_history_delayed previously,cat__credit_history_existing paid,cat__credit_history_no credits/all paid,cat__purpose_business,cat__purpose_domestic appliance,cat__purpose_education,cat__purpose_furniture/equipment,cat__purpose_new car,...,cat__foreign_worker_no,cat__foreign_worker_yes,remainder__checking_status,remainder__duration,remainder__credit_amount,remainder__installment_commitment,remainder__residence_since,remainder__age,remainder__existing_credits,remainder__num_dependents
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1,6,1169,4,4,67,2,1
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2,48,5951,2,2,22,1,1
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0,12,2096,2,3,49,1,2
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1,42,7882,2,4,45,1,2
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1,24,4870,3,4,53,2,2


### Opción 2: Label Encoding de sklearn

Podemos asignar un valor numérico a cada etiqueta en la misma columna usando Label Encoding de sklearn. Esto es útil si deseamos que cada categoría en una columna tenga un valor numérico sin crear múltiples columnas.

Sin embargo, debemos tener en cuenta que Label Encoding es recomendable cuando las categorías tienen un orden natural o cuando el modelo que usas no se verá afectado por esta codificación ordinal, ya que asigna valores numéricos a cada etiqueta de forma secuencial (por ejemplo, 0, 1, 2, etc.), lo cual puede no ser ideal si el modelo interpreta que existe un orden.

In [24]:
from sklearn.preprocessing import LabelEncoder

# Crear una copia de X para hacer los cambios
X_encoded = X.copy()

# Aplicar Label Encoding a cada columna categórica
label_encoders = {}  # Para guardar los encoders y poder revertir la codificación si es necesario
for col in X_encoded.select_dtypes(include=['category']).columns:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col])
    label_encoders[col] = le  # Guardar el encoder para revertir o transformar nuevos datos

# Mostrar las primeras filas del DataFrame transformado
X_encoded.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,1,6,1,6,1169,4,3,4,3,2,4,3,67,1,1,2,1,1,1,1
1,2,48,3,6,5951,2,0,2,0,2,2,3,22,1,1,1,1,1,0,1
2,0,12,1,2,2096,2,1,2,3,2,3,3,49,1,1,1,3,2,0,1
3,1,42,3,3,7882,2,1,2,3,1,4,1,45,1,0,1,1,2,0,1
4,1,24,2,4,4870,2,0,3,3,2,4,2,53,1,0,2,1,2,0,1


### Opción 3: pd.get_dummies de pandas

In [25]:
# Usar pd.get_dummies para convertir las columnas categóricas
X_encoded_df = pd.get_dummies(X, drop_first=True)

# Mostrar las primeras filas del DataFrame codificado
X_encoded_df.head()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,checking_status_1,checking_status_2,checking_status_3,...,property_magnitude_car,other_payment_plans_none,other_payment_plans_stores,housing_own,housing_rent,job_unemp/unskilled non res,job_unskilled resident,job_skilled,own_telephone_yes,foreign_worker_yes
0,6,1169,4,4,67,2,1,True,False,False,...,False,True,False,True,False,False,False,True,True,True
1,48,5951,2,2,22,1,1,False,True,False,...,False,True,False,True,False,False,False,True,False,True
2,12,2096,2,3,49,1,2,False,False,False,...,False,True,False,True,False,False,True,False,False,True
3,42,7882,2,4,45,1,2,True,False,False,...,False,True,False,False,False,False,False,True,False,True
4,24,4870,3,4,53,2,2,True,False,False,...,False,True,False,False,False,False,False,True,False,True


In [28]:
# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_encoded_df, y, test_size=0.3, random_state=42)

## Paso 4: Creación y Entrenamiento del Modelo Random Forest

Ahora vamos a crear nuestro modelo de Random Forest. Este es un modelo basado en muchos árboles de decisión (imaginemos muchos "árboles" que "votan" para decidir la respuesta).

In [29]:
# Crear el modelo de Random Forest
model = RandomForestClassifier(random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## Paso 5: Selección de los Hiperparámetros de Random Forest

* **n_estimators:** Este es el número de árboles que formarán el "bosque" (forest). Imagina que tenemos varios "árboles" (pequeños modelos de decisiones) que votan sobre la respuesta. Un número mayor de n_estimators hace que el modelo sea más robusto, pero también necesita más tiempo para entrenarse.

In [30]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

* **criterion:** Este es el criterio para medir si una división es buena. Imagina que en cada nodo del árbol, el modelo pregunta: "¿esta pregunta divide bien mis datos?" Los criterios más comunes son gini (medida de impureza) y entropy (basada en la ganancia de información). Ambos intentan que cada división sea lo más "pura" posible.

In [31]:
model = RandomForestClassifier(criterion='gini', random_state=42)

* **max_depth:** Esto limita la profundidad de cada árbol. Si es muy profundo, el árbol puede aprender hasta los detalles más pequeños de los datos (sobreajuste). En cambio, si limitamos la profundidad, el árbol puede enfocarse en patrones más generales. Imagina que cada nivel en el árbol representa una pregunta adicional para llegar a una decisión. Limitar max_depth es como decir: "No hagas demasiadas preguntas."

In [32]:
model = RandomForestClassifier(max_depth=10, random_state=42)

* **min_samples_split:** Este hiperparámetro define el número mínimo de datos que necesitamos en un grupo para dividirlo en más ramas. Por ejemplo, si min_samples_split es 10, el árbol solo dividirá un grupo si tiene al menos 10 datos. Esto evita que el árbol haga divisiones en grupos demasiado pequeños.

In [33]:
model = RandomForestClassifier(min_samples_split=5, random_state=42)

* **min_samples_leaf:** Este es el número mínimo de datos que se requieren en cada hoja del árbol (hoja = nodo final donde se toma una decisión). Si establecemos un número bajo, el árbol puede crear muchas hojas pequeñas, pero si lo establecemos alto, tendrá menos hojas y cada una con más datos.

In [34]:
model = RandomForestClassifier(min_samples_leaf=2, random_state=42)

* **max_features:** Este parámetro controla cuántas variables el modelo toma en cuenta al crear cada árbol. Si usamos todas las variables, cada árbol será más similar entre sí. Pero si limitamos max_features, cada árbol será un poco diferente, haciendo al bosque más variado y menos propenso al sobreajuste.

In [35]:
model = RandomForestClassifier(max_features='sqrt', random_state=42)

* **bootstrap:** Si bootstrap es True, el modelo seleccionará aleatoriamente ejemplos con reemplazo para construir cada árbol. Esto significa que un dato puede ser usado más de una vez en el entrenamiento de un solo árbol. Usar bootstrap=True ayuda a que cada árbol vea diferentes datos y no dependa solo de una parte del conjunto de entrenamiento.

In [36]:
model = RandomForestClassifier(bootstrap=True, random_state=42)

## Paso 6: Entrenar el Modelo

Ahora que entendemos cada parámetro, entrenaremos el modelo con algunos de ellos ajustados.

In [37]:
# Entrenar el modelo con hiperparámetros ajustados
model = RandomForestClassifier(
    n_estimators=150,
    max_depth=8,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    bootstrap=True,
    random_state=42
)

model.fit(X_train, y_train)

RandomForestClassifier(max_depth=8, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=150, random_state=42)

## Paso 7: Evaluar el Modelo

Finalmente, veamos cómo se desempeña el modelo en el conjunto de prueba:

In [38]:
# Hacer predicciones y evaluar la precisión
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy * 100:.2f}%')

Precisión del modelo: 74.67%
